# Reading catalog - LSST DP0 from Rubin Science Plataform

Uses the Table Access Protocol. 
<a href="http://www.ivoa.net/documents/TAP">Doc for TAP

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas

from lsst.rsp import get_tap_service

In [ ]:
pandas.set_option('display.max_rows', 20)

In [ ]:
service = get_tap_service()

assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

## Diving in the schemas to see what we have

In [ ]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query).to_table()

results

In [ ]:
query = "SELECT * FROM tap_schema.tables WHERE tap_schema.tables.schema_name = 'dp02_dc2_catalogs' order by table_index ASC"

results = service.search(query).to_table()
results

## Get columns for DP0 Objects

In [ ]:
results = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns WHERE table_name = 'dp02_dc2_catalogs.Object'")
results.to_table().to_pandas()

## Get bands for objects - circle and limiting by r band

In [ ]:
max_rec = 10
use_center_coords = "62, -37"
use_radius = "1.0"

In [ ]:
bands = ['g', 'i', 'r', 'u', 'y', 'z']

mags = ""
for band in bands:
    mags+= f"scisql_nanojanskyToAbMag({band}_cModelFlux) AS mag_{band}_cModel, {band}_cModelFluxErr, "

columns_query = f"objectId, coord_ra, coord_dec, {mags}detect_isPrimary, r_extendedness "

r_extendedness = 1 for extended sources, 0 for point sources. Measured on r-band. So for galaxies set 1 and stars 0

In [ ]:
query = "SELECT " + columns_query + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) > 17.0 " + \
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 23.0 "
print(query)

### Sync

memory_usage = deep get memory usage iuncluding object datatype

In [ ]:
%%time
results = service.search(query, maxrec=max_rec).to_table().to_pandas()
results.info(memory_usage="deep")

In [ ]:
results

### Async

In [ ]:
job = service.submit_job(query)

In [ ]:
job.run()

In [ ]:
%%time
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
%%time
results = job.fetch_result().to_table().to_pandas()
results.info(memory_usage="deep")

In [ ]:
results.query("mag_g_cModel == np.nan")

In [ ]:
help(results)

In [ ]:
del results